In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [ ]:
cd /content/gdrive/MyDrive/XAI/

/content/gdrive/MyDrive/XAI


In [ ]:
#from random import shuffle
!pip install folium
import folium
import pickle
import math
from tqdm import tqdm, trange
import pandas as pd
import numpy as np
import keras
import random as rn
import tensorflow as tf
import torch
import torch.optim as optim
import torch.nn as nn
from torch.nn.parameter import Parameter
from torch.nn.modules.module import Module
import torch.nn.functional as F
#from torch_geometric.nn.dense import DenseGCNConv
from keras.layers import Activation, Dense
from numpy import array as narray
from numpy import inf, median
from tensorflow.keras.optimizers import SGD,Adagrad, Adam, RMSprop

import easydict
import os
#import cupy #런타임 gpu 사용시 
import argparse
import dill
import matplotlib.pyplot as plt
from tqdm import tqdm
from math import atan
#런타임gpu사용시 
#from cupy import zeros, full, where, identity, divide, add, array, ndarray, asarray, asnumpy, stack, vstack, subtract, newaxis, multiply, ones, less, minimum, copy, concatenate, exp, repeat, tile, matmul, isnan, sqrt, log, NaN#, nanmean

import time
from queue import Queue
from sklearn.metrics import mean_squared_error
#geopandas
!pip install geopandas
import geopandas as gpd
import requests
from bs4 import BeautifulSoup
from sklearn.preprocessing import MinMaxScaler


     |████████████████████████████████| 1.0 MB 21.2 MB/s 
     |████████████████████████████████| 16.7 MB 51.2 MB/s 
     |████████████████████████████████| 6.3 MB 24.5 MB/s 


In [ ]:
#1차 전처리 
def InfoMatrix(g) :
    adj = np.zeros((g.shape[0], g.shape[0])) # adj 초기화
    for i in range(g.shape[0]) :
        idx = g[g['dst'] == g.at[i, 'src']].index.tolist() #연결이 되어 있으면 idx로 인덱스 정보 추출
        idx.extend(g[g['src'] == g.at[i, 'dst']].index.tolist())
        #print('IDS shape check : ',np.shape(idx))
        adj[i, idx] = 1 # 인접행렬을 생성
    return adj
    
def BaseInfo() : # 데이터 로드
    speed = pickle.load(open('2018_5pclean_tMatrix.pkl', 'rb')) # 속도 데이터 로드
    g = pickle.load(open('Graph_info.pkl', 'rb')).sort_values(by=['str_id']).reset_index(drop = True) # 인접관계 정보 데이터 로드
 
    # End Point 변환
    g.loc[g[g['dst'].isin(['힐튼'])].index,'dst'] = '도동삼거리'
    g.loc[g[g['dst'].isin(['힐튼'])].index,'dist'] = 322
    
    speed.drop(['3201500100'], axis = 1, inplace = True) # 기후 정보가 없는 링크 드랍
    g = g[g['str_id'] != '3201500100'].sort_values(by=['str_id']).reset_index(drop = True)
    g = g.sort_values(by=['str_id']).reset_index(drop = True)

    linklist = g['str_id'].values # 링크 ID 추출
    dist = g['dist'].astype(int).values # 거리 데이터 추출
    
    #adj = InfoMatrix(g) # 인접행렬 adj랑, cost행렬 추출
    dist = np.asarray(dist)
    #speed = np.asarray(speed.values)
    speed = np.asarray(speed.values).astype('float32')
   
    hopadj = pickle.load(open('tg1_hopadj.picle', 'rb'))
    lnewspeed = pickle.load(open('lnewspeed.picle', 'rb')) #local 노드별  scaled 된 속도 데이터 로드
    gnewspeed = pickle.load(open('gnewspeed.picle', 'rb')) #global하게  scaled 된 속도 데이터 로드
    return hopadj, g, speed, lnewspeed, gnewspeed


#hop count 계산 -> BFS알고리즘사용
def bfs(adj, start_node):
  x= len(adj)
  visit=[0]*x
  ch = [0]*x
  q = Queue()
  for s in np.where(adj[start_node]==1)[0]:
    q.put(s)
    visit[s]=1
    ch[s]=1
  while q.qsize()>0 :
    node = q.get()
    for nextN in np.where(adj[node]==1)[0]:
      if nextN != start_node and ch[nextN]==0:
        ch[nextN]=1
        q.put(nextN)
        visit[nextN]=visit[node]+1          
  return visit

def make_hopadj(adj): #hop count matrix 생성
  tg1_hopadj=[]
  for i in range(len(adj)):
    tg1_hopadj.append(bfs(adj, i))

  with open('tg1_hopadj.picle', 'wb') as f:
    pickle.dump(tg1_hopadj, f)

In [ ]:
'''
* fn1 : make_features(hopadj,speed, obj, tg)
dnn input을 만드는 함수로 obj가 test 인지 train인지에 맞추어서 input 생성하는 함수
input :
  hopadj = hop count 정보 
  speed = feature 만들기 위한 데이터 
  obj = list of 'test' OR 'train' index
  tg = target node idx

output :2차원( #train or test , #hop5이내 + 3)
obj의 hours에서 tg의 features

* fn2 : make_output(speed, obj, tg)
지도 학습이기에 ouput값이 필요, dnn의 ouput을 생성하는 함수

* fn3 : make_features_simulation(hopadj,speed, t, tg)
시뮬레이션 시 필요한 input 생성하는 함수
'''
def make_features(hopadj,speed, obj, tg): 
  
  #print(np.shape(hopadj))#(4670, 4670)
  check = True
  input_tg=[]
  for t in (obj): # (실험용) 노드에 대해서
    features=[] #(#hop5노드들 + 시각 + 요일,) 으로 features 표현 : 
    features.append(speed[t][tg].item()) # 해당 노드의 t시간대 속도
    for cnt in range(1,6): #hop count 1~5 까지
      hop_idx =  np.where(np.array(hopadj[tg])==cnt)[0]      
      #print(hop_idx)
      features.extend(speed[t][hop_idx])
    features.append( ( t%24 +1 )/24) #시각
    features.append( ( (t//24)%7 + 1 ) /7 ) #요일
    input_tg.append(features)
  
  if check:
    check=False
    #print('input_tg shape : (pred:  (#train or test , hop5이내 + 3)) vs ',np.shape(input_tg))# 2차원 (#train or test , hop5이내 갯수 + 3(자기자신속도,시각, 요일))

  return input_tg


def make_output(speed, obj, tg): #t시간대의 output --> t+1의 값 
  output_tg = []  
  for t in (obj):
    output_tg.append(speed[t+1][tg]) # 특정 시간 t에서의 전체 노드의 speed
  #print('output_tg shape : (pred:  (#train or test ) vs', np.shape(output_tg )) #(train or test,)
  return output_tg  

def make_features_simulation(hopadj,speed, t, tg): 
  
  feature=[] #(#hop5노드들 + 시각 + 요일,) 으로 features 표현 : 
  feature.append(speed[t][tg].item()) # 해당 노드의 t시간대 속도
 
  for cnt in range(1,6): #hop count 1~5 까지
    hop_idx =  np.where(np.array(hopadj[tg])==cnt)[0]      
    feature.extend(speed[t][hop_idx])
  feature.append( ( t%24 +1 )/24) #시각
  feature.append( ( (t//24)%7 + 1 ) /7 ) #요일
  
  return feature    

In [ ]:
def RMSE(y_test, y_predict): #판단 지표
    return np.sqrt(mean_squared_error(y_test, y_predict))

def MAPE(y_test, y_predict): #판단 지표
    return np.mean(np.abs((y_test-y_predict)/y_test))*100  

In [ ]:
if __name__=='__main__':    
  #Feature scaling  (x-mean)/(std) -> newspeed.picle로 저장해둠  
 
  #loading data
  hopadj, graphinfo, speed, lnewspeed, gnewspeed = BaseInfo() 
  
  #np.where조건식 2개 거는 방법
  ##idx0 =  np.where( (np.array(hopadj[0]) >=1) & (np.array(hopadj[0])<=5) )[0] 
  #adj 정규화  https://math.stackexchange.com/questions/3295135/how-is-the-adjacency-matrix-of-a-directed-graph-normalized
  
  ##########################################################################################
  #Step0) test train split
  ################################
  days = [31,28,31,30,31,30,31,31,30,31,30,31] #days of a year e.g) Jan: 31days, Feb: 28days, Mar: 31days ..
  for i in range(12):
    if i==0:
      pass
    else:
      days[i]+=days[i-1]

  month_test=[1,4,7,10]#Test: [2 Feb,5 May,8 Aug,11 Nov]
  hour_train = []
  hour_test = []

  #Split train & test data
  for i in range(12):
    if i == 0:
      tmp = range(0, days[i]*24)
    else:
      tmp = range(days[i-1] * 24, days[i]*24)
    if i in month_test:
      hour_test.extend(tmp)
    else:
      hour_train.extend(tmp)
  hour_train.pop() #NO next(t+1) data for a last day, so pop()
  print('hour_train len: ',len(hour_train))
  print('hour_test len : ', len(hour_test))

  ##########################################################################################
  #Step1) Set Base road        # e.g)'여의나루로','역삼로','방배로' road_name
  ####################################################################
  road_name = '여의나루로' # ****Set road namae****

  ginfo = pd.DataFrame(graphinfo)
  tg1 = ginfo[ginfo['road'].str.contains(road_name)]
  tg1idx = tg1.index.tolist()  
  print('tg1 {} idx : {}'.format(road_name, tg1idx)) #여의나루로 tg1idx: [4629, 4630, 4631, 4632, 4640, 4641] 
  print(tg1) #확인용                                  # --> 6 base roads in 'Yeoyinaru-Ro' -> each road is called 'tg' in code

  #globally하게 scaled된 speed data 사용!
  newspeed = gnewspeed
  scaled = 'G'
  ####################################
  tg1idx = tg1idx[4:5]

  for tg in tg1idx:

     #generate model for each base node
    print('** Current : {} out of {}'.format(tg,tg1idx)) 
    input_train = np.asarray(make_features(hopadj,newspeed, hour_train,tg)).astype('float32') 
    output_train = np.asarray(make_output(newspeed, hour_train, tg)).astype('float32')

    input_test = np.asarray(make_features(hopadj, newspeed, hour_test,tg)).astype('float32')
    output_test = np.asarray(make_output(newspeed, hour_test, tg)).astype('float32')

  input_train_df = pd.DataFrame(input_train, columns = ['base_id','3190','3219','3223','3244','4629',
           '2978','2983','2984','3156','3158','3187','3191','3199','3218','3222','3245','3254','4630','4631',
           '398','2081','2083','2976','2979','2982','2985','3126','3128','3134','3136','3153','3157','3159','3167','3175','3186','3198','3230','3255','3258','4632','4641',
           '393','395','397','445','470','2082','2084','2085','2087','2145','2974','2977','3114','3115','3116','3117','3123','3127','3129','3135','3137','3152','3166','3174','3179','3231','3259','3268','3352',
           '176','389','392','394','446','455','456','469','477','2086','2088','2091','2093','2146','2147','2148','2152','2158','2164','2970','2975','3122','3178','3185','3211','3261','3269','3278','3294','3295','3344','3351','3353','3354','3355',
           'clock_time','day']) 

  output_train_df = pd.DataFrame(output_train, columns = ['(t+1)speed'])

  input_test_df = pd.DataFrame(input_test, columns = ['base_id','3190','3219','3223','3244','4629',
           '2978','2983','2984','3156','3158','3187','3191','3199','3218','3222','3245','3254','4630','4631',
           '398','2081','2083','2976','2979','2982','2985','3126','3128','3134','3136','3153','3157','3159','3167','3175','3186','3198','3230','3255','3258','4632','4641',
           '393','395','397','445','470','2082','2084','2085','2087','2145','2974','2977','3114','3115','3116','3117','3123','3127','3129','3135','3137','3152','3166','3174','3179','3231','3259','3268','3352',
           '176','389','392','394','446','455','456','469','477','2086','2088','2091','2093','2146','2147','2148','2152','2158','2164','2970','2975','3122','3178','3185','3211','3261','3269','3278','3294','3295','3344','3351','3353','3354','3355',
           'clock_time','day']) 

  output_test_df = pd.DataFrame(output_test, columns = ['(t+1)speed']) 

  from sklearn.linear_model import Lasso
  model = Lasso(alpha=1.0)
  model.fit(input_train_df,output_train_df)
        
  ii = model.predict(input_test_df)
  print(np.shape(ii))
     
  ii = ii*130+0.74
  oo = output_test*130+0.74
      
  print('Slope: %.2f'% model.coef_[0])
  print('Intercept: %.2f'% model.intercept_)
  print('실제 속도로 본 RMSE :  ',RMSE(oo, ii)) # 실제, predict 

  from sklearn.metrics import mean_absolute_error
  print("MAE:",mean_absolute_error(oo,ii))

  print("MAPE:",MAPE(oo,ii))
 
  #model.save_model("model_sklearn_xgb.json"+scaled+str(tg))


hour_train len:  5879
hour_test len :  2880
tg1 여의나루로 idx : [4629, 4630, 4631, 4632, 4640, 4641]
          str_id     src     dst   road direction dist
4629  3201505000   문화방송앞    여의도역  여의나루로        하행  485
4630  3201505100    여의도역   문화방송앞  여의나루로        상행  485
4631  3201505200    여의도역   윤중초교앞  여의나루로        하행  398
4632  3201505300   윤중초교앞    여의도역  여의나루로        상행  398
4640  3201506200  여의중고교앞   문화방송앞  여의나루로        하행  455
4641  3201506300   문화방송앞  여의중고교앞  여의나루로        상행  455
** Current : 4640 out of [4640]
(2880,)
Slope: 0.00
Intercept: 0.16
실제 속도로 본 RMSE :   4.9031816
MAE: 3.6561658
MAPE: 16.040632128715515
